# Wrangle Data
##  Project : We Rate Dogs 

My goal in this wrangling act is to gather, assess and cleaning dataset from 3 different sources to finally be able to consolidate all this data into a single dataset to create interesting and trustworthy analyses and visualizations.

### Table of Content
<ul>
    <li><a href="#Data_Gathering">I. Data Gathering</a></li>
    <ul>
        <li><a href="#twit_arch">1. twitter_archive_enhanced.csv</a></li>
    <li><a href="#img_pred">2.  image_predictions.tsv Programmatically download</a></li>
    <li><a href="#tweet_metrics">3.tweet_json.txt</a></li>
</ul>
    <li><a href="#Data_Assessment">II. Data Assessment</a></li>
    <li><a href="#Data_Cleaning">III.Data Cleaning</a></li>
       <ul>
            <li><a href="#cleaning">1. Cleaning</a></li>
           <ul>
                    <li><a href="#clean1">A. Wrong Datatypes for timestamp column</a></li>
                    <li><a href="#clean2">B. Only original tweet (no retweet or reply)</a></li>
               <li><a href="#clean3">C. Wrong ratings score</a></li>
               <li><a href="#clean4">D. Delete denominator > 10</a></li>
               <li><a href="#clean5">E. Absurd numerator ratings left</a></li>
               <li><a href="#clean6">F. HTML in Source column</a></li>
               <li><a href="#clean7">G. Name column inconsistencies</a></li>
                    <li><a href="#clean8">H. standardization of dog breed names</a></li>
                </ul>      
            <li><a href="#tidy">2. Tydinness</a></li>
                <ul>
                    <li><a href="#tidy1">A. Stage dog column</a></li>
                    <li><a href="#tidy2">B. Delete the retweet and reply columns from archive</a></li>
                    <li><a href="#tidy3">C. Tydinnes issues of image_prediction_clean</a></li>
                </ul>
        </ul>
    <li><a href="#merge">IV. Merging column</a></li>
    <li><a href="#store">V. Store twitter_archive_master</a></li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json
import csv

<a id='Data_Gathering'></a>
#                                     I. Data Gathering

<a id='twit_arch'></a>
## 1. twitter_archive_enhanced.csv

twitter_archive_enhanced.csv has been manually uploaded to the notebook

#### Read and visualize twitter-archive-enhanced.csv

In [2]:
twit_arch = pd.read_csv('twitter-archive-enhanced.csv')
twit_arch.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [3]:
twit_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id='img_pred'></a>
## 2. image_predictions.tsv Programmatically download

#### Download, Read, visualize image-predictions.tsv

In [4]:
urls = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv']

for url in urls:
    response = requests.get(url)
    with open (os.path.join(url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)
        
img_pred = pd.read_csv('image-predictions.tsv', sep='\t')
img_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


<a id='tweet_metrics'></a>
## 3. tweet_json.txt 

#### Calling & accessing Twitter API using Tweepy

In [5]:
consumer_key = 'zlISSzjLNrpDoNxXz2MiEdCnX'
consumer_secret = 'cjsHFgxdRyRf1lK8mbgkxxFBxGzuNThciGME6RlvAfbKdlkAwj'
access_token = '2729738915-AVLRkroHmQBJuL7egsOXpHyldhhZ9VDw860h54y'
access_secret = 'qnWEA7bV5RMLoSNiY22A0JK0vCQmNNPwGAMOVssianbab'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)

#### Retrieve tweet ID Status, retweet count, and favorite count into tweet_json.txt

In [9]:
tweet_ok = []
tweet_error = []
for tweet_id in twit_arch['tweet_id']:
    try:
        tweet_ok.append(api.get_status(tweet_id, tweet_mode= 'extended'))
    except Exception as e:
         tweet_error.append(tweet_id)

Rate limit reached. Sleeping for: 410
Rate limit reached. Sleeping for: 724
Rate limit reached. Sleeping for: 702


In [13]:
print("The list of tweets" ,len(tweet_ok))
print("The list of tweets no found" , len(tweet_error))

The list of tweets 2331
The list of tweets no found 25


In [23]:
tweet_dicts = []
for each_json_tweet in tweet_ok:
    tweet_dicts.append(each_json_tweet)

In [24]:
with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(tweet_ok, indent=4))

In [5]:
#identify information of interest from JSON dictionaries in txt file
#and put it in a dataframe called tweet JSON
tweet_stat = []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_tweet = json.load(json_file)
    for each_dictionary in all_tweet:
        tweet_id = each_dictionary['id']
        fav = each_dictionary['favorite_count']
        rt = each_dictionary['retweet_count']

        tweet_stat.append({'tweet_id': str(tweet_id),
                             'fav': int(fav),
                             'rt': int(rt)
                            })
        tweet_metrics = pd.DataFrame(tweet_stat, columns = ['tweet_id', 'fav','rt'])

In [6]:
tweet_metrics.head()

,tweet_id,fav,rt
0,892420643555336193,36349,7733
1,892177421306343426,31320,5727
2,891815181378084864,23592,3788
3,891689557279858688,39654,7901
4,891327558926688256,37849,8520


<a id='Data_Assessment'></a>
#                                     II. Data Assessment

In [7]:
twit_arch.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


### Visually :

The following columns have missing value :

    - in_reply_to_status_id
    - in_reply_to_user_id
    - retweeted_status_id
    - retweeted_status_user_id
    - retweeted_status_timestamp

In [8]:
twit_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
twit_arch[twit_arch.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [10]:
twit_arch[twit_arch.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [11]:
twit_arch[twit_arch.expanded_urls.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
75,878281511006478336,NaN,NaN,2017-06-23 16:00:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Shadow. In an attempt to reach maximum zo...,NaN,NaN,NaN,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
76,878057613040115712,NaN,NaN,2017-06-23 01:10:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Emmy. She was adopted today. Massive r...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
98,873213775632977920,NaN,NaN,2017-06-09 16:22:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sierra. She's one precious pupper. Abs...,NaN,NaN,NaN,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
126,868552278524837888,NaN,NaN,2017-05-27 19:39:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Cooper. His expression is the sam...,NaN,NaN,NaN,"https://www.gofundme.com/3ti3nps,https://twitt...",12,10,Cooper,None,None,None,None
135,866450705531457537,NaN,NaN,2017-05-22 00:28:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jamesy. He gives a kiss to every other...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/866450705...,13,10,Jamesy,None,None,pupper,None
136,866334964761202691,NaN,NaN,2017-05-21 16:48:45 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Coco. At first I thought she was a clo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None


In [12]:
twit_arch[twit_arch.expanded_urls.isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
185,856330835276025856,NaN,NaN,2017-04-24 02:15:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Jenna_Marbles: @dog_rates Thanks for ratin...,8.563302e+17,66699013.0,2017-04-24 02:13:14 +0000,NaN,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None


In [13]:
twit_arch.name.value_counts()

None         745
a             55
Charlie       12
Cooper        11
Lucy          11
Oliver        11
Tucker        10
Lola          10
Penny         10
Bo             9
Winston        9
Sadie          8
the            8
Toby           7
Daisy          7
Bailey         7
an             7
Buddy          7
Leo            6
Scout          6
Koda           6
Jack           6
Bella          6
Milo           6
Stanley        6
Jax            6
Dave           6
Rusty          6
Oscar          6
Bentley        5
            ... 
Akumi          1
Edmund         1
Pepper         1
Brooks         1
Newt           1
Mojo           1
Charl          1
Kathmandu      1
Bauer          1
Koko           1
BeBe           1
Sunshine       1
Lipton         1
Pip            1
Obi            1
Taco           1
Lolo           1
Ralphé         1
Mike           1
Ralphie        1
Venti          1
Goose          1
Jeb            1
Humphrey       1
Maks           1
Keet           1
Sojourner      1
Karll         

Some Name are inconsistent, they only have small caps

In [14]:
num_aberation = twit_arch.rating_numerator.value_counts()
num_aberation.sort_index(ascending=False)

1776      1
960       1
666       1
420       2
204       1
182       1
165       1
144       1
143       1
121       1
99        1
88        1
84        1
80        1
75        2
60        1
50        1
45        1
44        1
27        1
26        1
24        1
20        1
17        1
15        2
14       54
13      351
12      558
11      464
10      461
9       158
8       102
7        55
6        32
5        37
4        17
3        19
2         9
1         9
0         2
Name: rating_numerator, dtype: int64

Some numerator are way above 10, according to the prject motivation it's seems normal but we'll check if their not corrupt

In [15]:
den_aberation = twit_arch.rating_denominator.value_counts()
den_aberation.sort_index(ascending=False)

170       1
150       1
130       1
120       1
110       1
90        1
80        2
70        1
50        3
40        1
20        2
16        1
15        1
11        3
10     2333
7         1
2         1
0         1
Name: rating_denominator, dtype: int64

Some denominator does not equal 10, we'll have to check those inconsistency

### Programmatically :

- It confirms our prior visual assessment, the `expanded_urls` columns also have missing values.

- The timestamp have the wrong data types (object instead of timestamp)

- While checking the null urls the last 4 columns seems untidy

- There is also a lot of None or non sens value (ex: a, an, bo, the, officially  ...) in name columns.

- Source columns have html in it

- The ratings in numerator and denominator columns are biased (should be 10) 

### `twit_arch` table

#### Missing Value for the Following Columns : 
    - in_reply_to_status_id
    - in_reply_to_user_id
    - retweeted_status_id
    - retweeted_status_user_id
    - retweeted_status_timestamp
    - expanded_urls
    
#### Wrong Data Types :
    - Timestamp
    
#### Cleanliness issue :
    - Check & fix error of rating_numerator
    - The rating_denominator should always be 10
    - HTML in Source column
    - None & non sens value in name columns

##### `img_pred` table

In [16]:
img_pred.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


Visually:

We see that the img_pred table is not tidy (p1 : p3 dog)

The p1,p2,p3 column have underscore and mix of capital & small caps

In [17]:
img_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [18]:
img_pred.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
cocker_spaniel                30
malamute                      30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
German_shepherd               20
Staffordshire_bullterrier     20
Siberian_husky                20
web_site                      19
Cardigan                      19
Maltese_dog                   18
Shetland_sheepdog             18
beagle                        18
Eskimo_dog                    18
teddy                         18
Shih-Tzu                      17
Rottweiler                    17
Lakeland_terrier              17
kuvasz                        16
Italian_greyhound             16
          

There is less record than the twit_arch df

Some Breed does not exist in real life such as scorpion, mailbox, pillow ...

Mix between capital and small capps letter


##### `tweet_metrics` table

In [19]:
tweet_metrics.head(3)

,tweet_id,fav,rt
0,892420643555336193,36349,7733
1,892177421306343426,31320,5727
2,891815181378084864,23592,3788


In [20]:
tweet_metrics.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
tweet_id    2331 non-null object
fav         2331 non-null int64
rt          2331 non-null int64
dtypes: int64(2), object(1)
memory usage: 54.7+ KB


#### Quality
### `twit_arch` table

#### Missing Value for the Following Columns : 
    - in_reply_to_status_id
    - in_reply_to_user_id
    - retweeted_status_id
    - retweeted_status_user_id
    - retweeted_status_timestamp
    
    - expanded_urls
    
#### Wrong Data Types :
    - Timestamp
    
#### Cleanliness issue :
    - The max rating_numerator should be 10
    - The rating_denominator should always be 10
    - HTML in Source column
    - None & non sens value in name columns

### `img_pred` table
    - less record than the twit_arch df
    - Some Breed does not exist in real life such as scorpion, mailbox, pillow ...
    - Mix between capital and small capps letter and underscore

#### Tidiness
- The retweet and reply are not interesting since they'll be empty
- doggo, floofer, pupper, puppo should be one column
- The prediction could be only 3 columns instead of 9
- Merge the twit_arch & tweet_metric

<a id='Data_Cleaning'></a>
#                                     III. Data Cleaning & Tydinness

<a id='cleaning'></a>
### I. Cleaning

### `twit_arch` table

In [21]:
twitter_archive_clean = twit_arch.copy()

In [22]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id='clean1'></a>
#### I. Wrong Datatypes for timestamp column

The timestamp column is an object instead of a proper timestamp

In [23]:
twitter_archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [24]:
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])

In [25]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64[ns](1

In [26]:
twitter_archive_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


<a id='clean2'></a>
#### II. Only original tweet (no retweet or reply)

As state during the project motivation section we have to delete all the rows how is not null in the following columns :

    - retweeted_status_id
    - retweeted_status_user_id
    - retweeted_status_timestamp

In [27]:
twitter_archive_clean = twitter_archive_clean.query("retweeted_status_id == 'NaN' and in_reply_to_user_id =='NaN'")

In [28]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2097 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2097 non-null datetime64[ns]
source                        2097 non-null object
text                          2097 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2094 non-null object
rating_numerator              2097 non-null int64
rating_denominator            2097 non-null int64
name                          2097 non-null object
doggo                         2097 non-null object
floofer                       2097 non-null object
pupper                        2097 non-null object
puppo                         2097 non-null object
dtypes: datetime64[ns](1), float

<a id='clean3'></a>
#### III. Wrong ratings score

Even if the project motivation specifies that's The fact that the rating numerators are greater than the denominators does not need to be cleaned some rating seems really overrated and need a check

In [29]:
twitter_score_status = twitter_archive_clean[['tweet_id','text','rating_numerator','rating_denominator']]
pd.set_option('display.max_colwidth', -1)
twitter_score_status.query("rating_denominator != 10")

,tweet_id,text,rating_numerator,rating_denominator
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,740373189193256964,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,722974582966214656,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,716439118184652801,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50
1228,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99,90
1254,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80,80
1274,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45,50


tweet_id : 666287406224695296 score 1/2 instead of 9/10

tweet_id : 740373189193256964 score 9/11 instead of 14/10

tweet_id : 682962037429899265 score 7/11 instead of 10/10

tweet_id : 716439118184652801 score 50/50 instead of 11/10

tweet_id : 722974582966214656 score 4/20 instead of 13/10

In [30]:
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 666287406224695296, ['rating_numerator','rating_denominator']] = 9, 10
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 740373189193256964, ['rating_numerator','rating_denominator']] = 14, 10
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 682962037429899265, ['rating_numerator','rating_denominator']] = 10, 10
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 716439118184652801, ['rating_numerator','rating_denominator']] = 11, 10
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 722974582966214656, ['rating_numerator','rating_denominator']] = 13, 10

In [31]:
twitter_score_status2 = twitter_archive_clean[['tweet_id','text','rating_numerator','rating_denominator']]
pd.set_option('display.max_colwidth', -1)
twitter_score_status2.query("rating_denominator != 10")

,tweet_id,text,rating_numerator,rating_denominator
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1228,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99,90
1254,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80,80
1274,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45,50
1351,704054845121142784,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60,50
1433,697463031882764288,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,44,40
1635,684222868335505415,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,121,110


<a id='clean4'></a>
#### IV.Delete denominator > 10

As seen right before, some denominator that fall under the recording error aren't equal to 10. those tweet doesn't have appropriate ratings, we are going to delete them.

In [32]:
twitter_archive_clean = twitter_archive_clean.query("rating_denominator == 10")
twitter_archive_clean['rating_denominator'].value_counts()

10    2085
Name: rating_denominator, dtype: int64

<a id='clean5'></a>
#### V. Absurd numerator ratings left

Let's check for some absurd ratings in the numerator

In [33]:
tweet_num_test = twitter_archive_clean.query("rating_numerator > 10")
tweet_num_test['rating_numerator'].value_counts()

12      486
11      414
13      288
14      39 
1776    1  
420     1  
75      1  
27      1  
26      1  
Name: rating_numerator, dtype: int64

Some numerator seems inconsistent let's check their ratings

In [34]:
tweet_num_test.query("rating_numerator > 25")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082849828864/photo/1,75,10,Logan,None,None,None,None
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034220126208/photo/1,27,10,Sophie,None,None,pupper,None
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277374128128/photo/1,1776,10,Atticus,None,None,None,None
1712,680494726643068929,NaN,NaN,2015-12-25 21:06:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680494726643068929/photo/1,26,10,None,None,None,None,None
2074,670842764863651840,NaN,NaN,2015-11-29 05:52:33,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",After so many requests... here you go.\n\nGood dogg. 420/10 https://t.co/yfAAo1gdeY,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670842764863651840/photo/1,420,10,None,None,None,None,None


Due to decimal some rating were recorded incorrectly. let's fix this by correcting the numerator ratings to the nearest whole number.

In [35]:
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 786709082849828864, ['rating_numerator','rating_denominator']] = 10, 10
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 778027034220126208, ['rating_numerator','rating_denominator']] = 11, 10
twitter_archive_clean.loc[twitter_archive_clean.tweet_id == 680494726643068929, ['rating_numerator','rating_denominator']] = 11, 10

In [36]:
twitter_archive_clean['rating_numerator'].value_counts()

12      486
10      438
11      416
13      288
9       153
8       98 
7       51 
14      39 
5       34 
6       32 
3       19 
4       15 
2       9  
1       4  
420     1  
1776    1  
0       1  
Name: rating_numerator, dtype: int64

The ratings now seems more consistent, note that the scores of 1776 and 420 are not recording errors but the ratings given in the tweet.

<a id='clean6'></a>
#### VI. HTML in Source column

Using regex, replace the HTML tag

In [37]:
twitter_archive_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None


In [38]:
twitter_archive_clean['source'][0]

'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'

In [39]:
twitter_archive_clean['source'] = twitter_archive_clean['source'].str.replace('<[^<]+?>', '')

In [40]:
twitter_archive_clean['source'].value_counts()

Twitter for iPhone     1952
Vine - Make a Scene    91  
Twitter Web Client     31  
TweetDeck              11  
Name: source, dtype: int64

<a id='clean7'></a>
#### VII. Name column inconsistencies

it has been observed, some inconsistence in the name column.
I'll first detect them and replace to "None" and then delete all the None name value.

In [41]:
twitter_archive_clean['name'].value_counts()

None         594
a            54 
Charlie      11 
Lucy         11 
Cooper       10 
Oliver       10 
Penny        9  
Tucker       9  
Sadie        8  
the          8  
Lola         8  
Winston      8  
Toby         7  
Daisy        7  
Bo           6  
Bailey       6  
Jax          6  
Oscar        6  
an           6  
Koda         6  
Bella        6  
Stanley      6  
Buddy        5  
Louis        5  
Dave         5  
Rusty        5  
Scout        5  
Bentley      5  
Milo         5  
Leo          5  
            ..  
Clyde        1  
Anakin       1  
Tedders      1  
Willow       1  
Humphrey     1  
Goose        1  
Grizzwald    1  
Venti        1  
Vinscent     1  
Jareld       1  
Roscoe       1  
Cermet       1  
Tanner       1  
Sora         1  
General      1  
Philbert     1  
Kathmandu    1  
Koko         1  
BeBe         1  
Sunshine     1  
Lipton       1  
Pip          1  
Obi          1  
Taco         1  
Lolo         1  
Paull        1  
Ralphé       1  
Bauer        1

In [42]:
twit_fake_name = twitter_archive_clean['name'].str.contains('^[a-z]')
twitter_archive_clean[twit_fake_name].name.value_counts()

a               54
the             8 
an              6 
one             4 
very            4 
just            3 
quite           3 
getting         2 
actually        2 
not             2 
space           1 
mad             1 
old             1 
such            1 
my              1 
unacceptable    1 
all             1 
his             1 
officially      1 
infuriating     1 
incredibly      1 
by              1 
light           1 
life            1 
Name: name, dtype: int64

Except the None value who is capitalized, all the other small caps name are inconsistent

In [43]:
fk_name = ['a','the', 'an', 'very','one', 'quite', 'just', 'actually', 'getting', 'not', 'old','space', 'all', 'this', 'my', 'such', 'unacceptable', 'officially', 'by', 'light' ,'incredibly', 'his','mad', 'life', 'infuriating']

for names in twitter_archive_clean['name']:
    if names in fk_name:
        twitter_archive_clean['name'][twitter_archive_clean['name'] == names] = 'None'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [44]:
twitter_archive_clean['name'].value_counts()

None         696
Lucy         11 
Charlie      11 
Oliver       10 
Cooper       10 
Penny        9  
Tucker       9  
Lola         8  
Sadie        8  
Winston      8  
Daisy        7  
Toby         7  
Bailey       6  
Bella        6  
Koda         6  
Jax          6  
Stanley      6  
Oscar        6  
Bo           6  
Milo         5  
Bentley      5  
Rusty        5  
Louis        5  
Chester      5  
Leo          5  
Dave         5  
Scout        5  
Buddy        5  
Sophie       4  
Chip         4  
            ..  
Kota         1  
Harper       1  
Harnold      1  
Godzilla     1  
Clyde        1  
Mike         1  
Bauer        1  
Ralphé       1  
General      1  
Jay          1  
Clarq        1  
Grizzwald    1  
Vinscent     1  
Jareld       1  
Roscoe       1  
Cermet       1  
Tanner       1  
Sora         1  
Philbert     1  
Paull        1  
Kathmandu    1  
Koko         1  
BeBe         1  
Sunshine     1  
Lipton       1  
Pip          1  
Obi          1  
Taco         1

In [45]:
twitter_archive_clean = twitter_archive_clean.query("name !='None'")
twitter_archive_clean['name'].value_counts()

Charlie      11
Lucy         11
Cooper       10
Oliver       10
Penny        9 
Tucker       9 
Sadie        8 
Lola         8 
Winston      8 
Daisy        7 
Toby         7 
Bailey       6 
Stanley      6 
Jax          6 
Bella        6 
Koda         6 
Bo           6 
Oscar        6 
Buddy        5 
Leo          5 
Milo         5 
Scout        5 
Chester      5 
Louis        5 
Dave         5 
Bentley      5 
Rusty        5 
Larry        4 
Reggie       4 
Archie       4 
            .. 
Harper       1 
Harnold      1 
Godzilla     1 
Clyde        1 
Anakin       1 
Venti        1 
Mike         1 
Bauer        1 
Philbert     1 
Clarq        1 
Grizzwald    1 
Vinscent     1 
Jareld       1 
Roscoe       1 
Cermet       1 
Tanner       1 
Sora         1 
General      1 
Kathmandu    1 
Ralphé       1 
Koko         1 
BeBe         1 
Sunshine     1 
Lipton       1 
Pip          1 
Obi          1 
Taco         1 
Lolo         1 
Paull        1 
Willem       1 
Name: name, Length: 929,

All the inconsistent value in name column has been deleted

### `img_pred` table

In [46]:
image_prediction_clean = img_pred.copy()

In [47]:
image_prediction_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


<a id='clean8'></a>
#### VIII. standardization of dog breed names 

##### Phase 1 : Replace Undersore

In order to have standardized dog breed names we are going to replace the undersore per a space
Using the replace method in p1, p2 & p3 column

In [48]:
for breed in image_prediction_clean[['p1','p2','p3']]:
    image_prediction_clean[breed] = image_prediction_clean[breed].str.replace('_',' ')

In [49]:
image_prediction_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh springer spaniel,0.465074,True,collie,0.156665,True,Shetland sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature pinscher,0.074192,True,Rhodesian ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian ridgeback,0.408143,True,redbone,0.360687,True,miniature pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


##### Phase 2 : capital letter for each words first letter

in the continuation of the objective of standardization of dog breed names we are going to replace in capital letter the first letter of each word present in the columns p1 p2 and p3.

Using the title() method

In [50]:
for breedcap in image_prediction_clean[['p1','p2','p3']]:
    image_prediction_clean[breedcap] = image_prediction_clean[breedcap].str.title()

In [51]:
image_prediction_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,0.596461,True,Malinois,0.138584,True,Bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback,0.408143,True,Redbone,0.360687,True,Miniature Pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


<a id='tidy'></a>
### II. Tydinnes

<a id='tidy1'></a>
#### I. Stage dog column

Melt the *doggo*, *floofer*, *pupper*, *puppo* columns to a *stage* column.

In [52]:
twitter_archive_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None


In [53]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 0 to 2325
Data columns (total 17 columns):
tweet_id                      1389 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1389 non-null datetime64[ns]
source                        1389 non-null object
text                          1389 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1389 non-null object
rating_numerator              1389 non-null int64
rating_denominator            1389 non-null int64
name                          1389 non-null object
doggo                         1389 non-null object
floofer                       1389 non-null object
pupper                        1389 non-null object
puppo                         1389 non-null object
dtypes: datetime64[ns](1), float

In [54]:
twitter_archive_clean2 = pd.melt(twitter_archive_clean, id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name'])
twitter_archive_clean2

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,variable,value
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,doggo,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,doggo,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,doggo,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,doggo,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,doggo,None
5,890971913173991426,NaN,NaN,2017-07-28 16:27:12,Twitter for iPhone,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,doggo,None
6,890609185150312448,NaN,NaN,2017-07-27 16:25:51,Twitter for iPhone,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,doggo,None
7,890240255349198849,NaN,NaN,2017-07-26 15:59:51,Twitter for iPhone,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255349198849/photo/1,14,10,Cassie,doggo,doggo
8,890006608113172480,NaN,NaN,2017-07-26 00:31:25,Twitter for iPhone,This is Koda. He is a South Australian deckshark. Deceptively deadly. Frighteningly majestic. 13/10 would risk a petting #BarkWeek https://t.co/dVPW0B0Mme,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890006608113172480/photo/1,https://twitter.com/dog_rates/status/890006608113172480/photo/1",13,10,Koda,doggo,None
9,889880896479866881,NaN,NaN,2017-07-25 16:11:53,Twitter for iPhone,This is Bruno. He is a service shark. Only gets out of the water to assist you. 13/10 terrifyingly good boy https://t.co/u1XPQMl29g,NaN,NaN,NaN,https://twitter.com/dog_rates/status/889880896479866881/photo/1,13,10,Bruno,doggo,None


In [55]:
stage_match = twitter_archive_clean2.query("variable == value")
stage_nomatch = twitter_archive_clean2.query("variable != value")

In [56]:
del stage_nomatch['variable']
del stage_match['variable']

In [57]:
stage_nomatch = stage_nomatch.drop_duplicates()
stage_match = stage_match.drop_duplicates()

In [58]:
stage_match.tweet_id.value_counts()

817777686764523521    2
801115127852503040    2
759793422261743616    2
785639753186217984    2
751950017322246144    1
681891461017812993    1
673697980713705472    1
685307451701334016    1
675006312288268288    1
686618349602762752    1
679503373272485890    1
828381636999917570    1
673919437611909120    1
819952236453363712    1
819004803107983360    1
811386762094317568    1
705975130514706432    1
750011400160841729    1
693095443459342336    1
826848821049180160    1
683142553609318400    1
675845657354215424    1
680473011644985345    1
686683045143953408    1
721001180231503872    1
867421006826221569    1
890240255349198849    1
703611486317502464    1
679777920601223168    1
760893934457552897    1
                     ..
878776093423087618    1
750868782890057730    1
719704490224398336    1
694329668942569472    1
681694085539872773    1
789903600034189313    1
871515927908634625    1
873213775632977920    1
781655249211752448    1
673612854080196609    1
8130969848233492

Most of the stage are indicated as None
4 dogs have 2 stages : 
                     -   817777686764523521 -> pupper       
                     -   801115127852503040 -> doggo
                     -   759793422261743616 -> doggo
                     -   785639753186217984 -> doggo


In [59]:
stage_match = stage_match.drop(270)
stage_match = stage_match.drop(3117)
stage_match = stage_match.drop(3311)
stage_match = stage_match.drop(3191)

In [60]:
twitter_archive_clean3 = pd.concat([stage_match, stage_nomatch])
twitter_archive_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1573 entries, 7 to 2053
Data columns (total 14 columns):
tweet_id                      1573 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1573 non-null datetime64[ns]
source                        1573 non-null object
text                          1573 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1573 non-null object
rating_numerator              1573 non-null int64
rating_denominator            1573 non-null int64
name                          1573 non-null object
value                         1573 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(6)
memory usage: 184.3+ KB


In [61]:
twitter_archive_clean3.tweet_id.value_counts()

749417653287129088    2
801115127852503040    2
819006400881917954    2
711008018775851008    2
689623661272240129    2
845306882940190720    2
672898206762672129    2
683481228088049664    2
786363235746385920    2
692417313023332352    2
884162670584377345    2
672481316919734272    2
692905862751522816    2
674737130913071104    2
685169283572338688    2
680145970311643136    2
681694085539872773    2
789903600034189313    2
750383411068534784    2
689905486972461056    2
832032802820481025    2
760893934457552897    2
673707060090052608    2
742150209887731712    2
821407182352777218    2
696713835009417216    2
672594978741354496    2
800388270626521089    2
826958653328592898    2
869596645499047938    2
                     ..
667453023279554560    1
741099773336379392    1
745712589599014916    1
708853462201716736    1
780858289093574656    1
735991953473572864    1
675111688094527488    1
669000397445533696    1
797236660651966464    1
777885040357281792    1
8258765121591869

In [62]:
twitter_archive_clean3 = twitter_archive_clean3.drop_duplicates(subset='tweet_id', keep="first")

In [63]:
twitter_archive_clean3.rename(columns={'value': 'stage'}, inplace=True)

In [64]:
twitter_archive_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 7 to 1388
Data columns (total 14 columns):
tweet_id                      1389 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1389 non-null datetime64[ns]
source                        1389 non-null object
text                          1389 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1389 non-null object
rating_numerator              1389 non-null int64
rating_denominator            1389 non-null int64
name                          1389 non-null object
stage                         1389 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(6)
memory usage: 162.8+ KB


In [65]:
twitter_archive_clean = twitter_archive_clean3

<a id='tidy2'></a>
#### II. Delete the retweet and reply columns from archive

In [66]:
del twitter_archive_clean['in_reply_to_status_id']
del twitter_archive_clean['in_reply_to_user_id']
del twitter_archive_clean['retweeted_status_user_id']
del twitter_archive_clean['retweeted_status_id']
del twitter_archive_clean['retweeted_status_timestamp']

In [67]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 7 to 1388
Data columns (total 9 columns):
tweet_id              1389 non-null int64
timestamp             1389 non-null datetime64[ns]
source                1389 non-null object
text                  1389 non-null object
expanded_urls         1389 non-null object
rating_numerator      1389 non-null int64
rating_denominator    1389 non-null int64
name                  1389 non-null object
stage                 1389 non-null object
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 108.5+ KB


<a id='tidy3'></a>
#### III. Tydinnes issues of image_prediction_clean

Here we have 9 Columns to represent the 3 predictions,
I would like to replace them in 4 columns to select only the best prediction and know which one it was.
the four columns would be as follows: 
    - # predictions
    - dog breed name
    - confidence of prediction
    - boolean of whether it's a dog or not.

In [68]:
image_prediction_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,0.596461,True,Malinois,0.138584,True,Bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback,0.408143,True,Redbone,0.360687,True,Miniature Pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [69]:
image_prediction_clean2 =  image_prediction_clean
image_prediction_clean2.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,0.596461,True,Malinois,0.138584,True,Bloodhound,0.116197,True


In [70]:
max_conf = image_prediction_clean2[["p1_conf", "p2_conf","p3_conf"]].max(axis=1)
image_prediction_clean2['confidence'] = max_conf

In [71]:
image_prediction_clean2

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,0.596461,True,Malinois,0.138584,True,Bloodhound,0.116197,True,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback,0.408143,True,Redbone,0.360687,True,Miniature Pinscher,0.222752,True,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True,0.560311
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese Mountain Dog,0.651137,True,English Springer,0.263788,True,Greater Swiss Mountain Dog,0.016199,True,0.651137
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,Box Turtle,0.933012,False,Mud Turtle,0.045885,False,Terrapin,0.017885,False,0.933012
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,Chow,0.692517,True,Tibetan Mastiff,0.058279,True,Fur Coat,0.054449,False,0.692517
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,Shopping Cart,0.962465,False,Shopping Basket,0.014594,False,Golden Retriever,0.007959,True,0.962465
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,Miniature Poodle,0.201493,True,Komondor,0.192305,True,Soft-Coated Wheaten Terrier,0.082086,True,0.201493


let's see if the first prediction is always the better

In [72]:
image_prediction_clean2['check'] = (image_prediction_clean2['p1_conf'] == image_prediction_clean2['confidence']).astype('object')

In [73]:
image_prediction_clean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 14 columns):
tweet_id      2075 non-null int64
jpg_url       2075 non-null object
img_num       2075 non-null int64
p1            2075 non-null object
p1_conf       2075 non-null float64
p1_dog        2075 non-null bool
p2            2075 non-null object
p2_conf       2075 non-null float64
p2_dog        2075 non-null bool
p3            2075 non-null object
p3_conf       2075 non-null float64
p3_dog        2075 non-null bool
confidence    2075 non-null float64
check         2075 non-null object
dtypes: bool(3), float64(4), int64(2), object(5)
memory usage: 184.5+ KB


In [74]:
image_prediction_clean2['check'].value_counts()

True    2075
Name: check, dtype: int64

The first prediction is always the better

In [75]:
image_prediction_clean2['breed'] = image_prediction_clean2.iloc[:,[3]]
image_prediction_clean2['isdog?'] = image_prediction_clean2.iloc[:,[5]]
del image_prediction_clean2['check']

In [76]:
image_prediction_clean2 = image_prediction_clean2.drop(['p1', 'p2','p3','p1_conf', 'p2_conf','p3_conf','p1_dog', 'p2_dog','p3_dog'], axis = 1)

In [77]:
image_prediction_clean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 6 columns):
tweet_id      2075 non-null int64
jpg_url       2075 non-null object
img_num       2075 non-null int64
confidence    2075 non-null float64
breed         2075 non-null object
isdog?        2075 non-null bool
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 83.2+ KB


In [78]:
image_prediction_clean = image_prediction_clean2

<a id='merge'></a>
### Merging column

In [79]:
image_prediction_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 6 columns):
tweet_id      2075 non-null int64
jpg_url       2075 non-null object
img_num       2075 non-null int64
confidence    2075 non-null float64
breed         2075 non-null object
isdog?        2075 non-null bool
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 83.2+ KB


In [80]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 7 to 1388
Data columns (total 9 columns):
tweet_id              1389 non-null int64
timestamp             1389 non-null datetime64[ns]
source                1389 non-null object
text                  1389 non-null object
expanded_urls         1389 non-null object
rating_numerator      1389 non-null int64
rating_denominator    1389 non-null int64
name                  1389 non-null object
stage                 1389 non-null object
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 108.5+ KB


In [81]:
tweet_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
tweet_id    2331 non-null object
fav         2331 non-null int64
rt          2331 non-null int64
dtypes: int64(2), object(1)
memory usage: 54.7+ KB


In [82]:
tweet_metrics['tweet_id'] = tweet_metrics['tweet_id'].astype(str).astype(int)

In [83]:
tweet_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
tweet_id    2331 non-null int64
fav         2331 non-null int64
rt          2331 non-null int64
dtypes: int64(3)
memory usage: 54.7 KB


In [84]:
twitter_archive_master = pd.merge(twitter_archive_clean, image_prediction_clean, on = 'tweet_id', how = 'left')

In [87]:
twitter_archive_master = pd.merge(twitter_archive_master, tweet_metrics, on = 'tweet_id', how = 'left')

In [88]:
twitter_archive_master

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,stage,jpg_url,img_num,confidence,breed,isdog?,fav_x,rt_x,fav_y,rt_y
0,890240255349198849,2017-07-26 15:59:51,Twitter for iPhone,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,https://twitter.com/dog_rates/status/890240255349198849/photo/1,14,10,Cassie,doggo,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,1.0,0.511319,Pembroke,True,29978.0,6725.0,29978.0,6725.0
1,884162670584377345,2017-07-09 21:29:42,Twitter for iPhone,Meet Yogi. He doesn't have any important dog meetings today he just enjoys looking his best at all times. 12/10 for dangerously dapper doggo https://t.co/YSI00BzTBZ,https://twitter.com/dog_rates/status/884162670584377345/photo/1,12,10,Yogi,doggo,https://pbs.twimg.com/media/DEUtQbzW0AUTv_o.jpg,1.0,0.707046,German Shepherd,True,19210.0,2740.0,19210.0,2740.0
2,871515927908634625,2017-06-04 23:56:03,Twitter for iPhone,This is Napolean. He's a Raggedy East Nicaraguan Zoom Zoom. Runs on one leg. Built for deception. No eyes. Good with kids. 12/10 great doggo https://t.co/PR7B7w1rUw,"https://twitter.com/dog_rates/status/871515927908634625/photo/1,https://twitter.com/dog_rates/status/871515927908634625/photo/1",12,10,Napolean,doggo,https://pbs.twimg.com/media/DBg_HT9WAAEeIMM.jpg,2.0,0.974781,Komondor,True,19134.0,3228.0,19134.0,3228.0
3,869596645499047938,2017-05-30 16:49:31,Twitter for iPhone,This is Scout. He just graduated. Officially a doggo now. Have fun with taxes and losing sight of your ambitions. 12/10 would throw cap for https://t.co/DsA2hwXAJo,"https://twitter.com/dog_rates/status/869596645499047938/photo/1,https://twitter.com/dog_rates/status/869596645499047938/photo/1",12,10,Scout,doggo,https://pbs.twimg.com/media/DBFtiYqWAAAsjj1.jpg,1.0,0.955156,Chihuahua,True,15209.0,2913.0,15209.0,2913.0
4,846514051647705089,2017-03-28 00:07:32,Twitter for iPhone,This is Barney. He's an elder doggo. Hitches a ride when he gets tired. Waves goodbye before he leaves. 13/10 please come back soon https://t.co/cFAasDXauK,"https://twitter.com/dog_rates/status/846514051647705089/photo/1,https://twitter.com/dog_rates/status/846514051647705089/photo/1,https://twitter.com/dog_rates/status/846514051647705089/photo/1",13,10,Barney,doggo,https://pbs.twimg.com/media/C79sB4xXwAEvwKY.jpg,2.0,0.650003,Golden Retriever,True,44360.0,11451.0,44360.0,11451.0
5,845397057150107648,2017-03-24 22:08:59,Twitter for iPhone,"Say hello to Mimosa. She's an emotional support doggo who helps her owner with PTSD. 13/10, but she needs your help\n\nhttps://t.co/L6mLzrd7Mx https://t.co/jMutBFdw5o","https://www.gofundme.com/help-save-a-pup,https://twitter.com/dog_rates/status/845397057150107648/photo/1,https://twitter.com/dog_rates/status/845397057150107648/photo/1",13,10,Mimosa,doggo,https://pbs.twimg.com/media/C7t0IzLWkAINoft.jpg,1.0,0.394404,Dandie Dinmont,True,7608.0,1790.0,7608.0,1790.0
6,836753516572119041,2017-03-01 01:42:39,Twitter for iPhone,This is Meera. She just heard about taxes and how much a doghouse in a nice area costs. Not pupared to be a doggo anymore. 12/10 https://t.co/GZmNEdyoJY,https://twitter.com/dog_rates/status/836753516572119041/photo/1,12,10,Meera,doggo,https://pbs.twimg.com/media/C5y-4VwWcAIcaoj.jpg,1.0,0.936882,Mortarboard,False,19413.0,4657.0,19413.0,4657.0
7,832998151111966721,2017-02-18 17:00:10,Twitter for iPhone,"This is Rhino. He arrived at a shelter with an elaborate doggo manual for his new family, written by someone who will always love him. 13/10 https://t.co/QX1h0oqMz0","https://twitter.com/dog_rates/status/832998151111966721/photo/1,https://twitter.com/dog_rates/status/832998151111966721/photo/1",13,10,Rhino,doggo,https://pbs.twimg.com/media/C49nZavUYAEJjGw.jpg,1.0,0.539036,Boxer,True,13373.0,2218.0,13373.0,2218.0
8,832273440279240704,2017-02-16 17:00:25,Twitter for iPhone,Say hello to Smiley. He's a blind therapy doggo h

In [89]:
twitter_archive_master = twitter_archive_master.drop(['jpg_url', 'img_num', 'fav_y', 'rt_y'], axis = 1)
twitter_archive_master.rename(columns={'fav_x':'favorite', 'rt_x':'retweet'}, inplace=True)

Let's check if the twitter_archive_master table is clean

In [91]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 0 to 1388
Data columns (total 14 columns):
tweet_id              1389 non-null int64
timestamp             1389 non-null datetime64[ns]
source                1389 non-null object
text                  1389 non-null object
expanded_urls         1389 non-null object
rating_numerator      1389 non-null int64
rating_denominator    1389 non-null int64
name                  1389 non-null object
stage                 1389 non-null object
confidence            1348 non-null float64
breed                 1348 non-null object
isdog?                1348 non-null object
favorite              1382 non-null float64
retweet               1382 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(7)
memory usage: 162.8+ KB


As expected the column merged in the prior step does not exactly retrieve all the tweet_id.
We're gonna to delete the missing value of the last 5 columns

In [94]:
twitter_archive_master = twitter_archive_master.dropna(axis=0, subset=['confidence'])
twitter_archive_master = twitter_archive_master.dropna(axis=0, subset=['favorite'])

In [95]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1341 entries, 0 to 1388
Data columns (total 14 columns):
tweet_id              1341 non-null int64
timestamp             1341 non-null datetime64[ns]
source                1341 non-null object
text                  1341 non-null object
expanded_urls         1341 non-null object
rating_numerator      1341 non-null int64
rating_denominator    1341 non-null int64
name                  1341 non-null object
stage                 1341 non-null object
confidence            1341 non-null float64
breed                 1341 non-null object
isdog?                1341 non-null object
favorite              1341 non-null float64
retweet               1341 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(7)
memory usage: 157.1+ KB


The twitter_archive_master seems now to be cleaner, it's time to store it into a csv

<a id='store'></a>
### Store twitter_archive_master

In [96]:
twitter_archive_master.to_csv('twitter_archive_master.csv', index = False)

# To see the Analysis project please go to Analysis_Act.ipynb